In [1]:
import os, cv2
import splitfolders
import shutil

In [2]:
# Define your input and output folder paths
input_folder = '/home/sowmya/AgSolaire/data'
output_folder = '/home/sowmya/AgSolaire/train_val_test_split'

# Check if the output folder exists and delete it if it does
try:
    shutil.rmtree(output_folder)
    print(f"Removed existing folder: {output_folder}")
except FileNotFoundError:
    print(f"Output folder not found, creating a new one: {output_folder}")

# Perform the train/validation/test split
splitfolders.ratio(
    input_folder, 
    output=output_folder, 
    seed=1337, 
    ratio=(0.8, 0.1, 0.1), 
    move=False
)

Removed existing folder: /home/sowmya/AgSolaire/train_val_test_split


In [ ]:
from ultralytics import YOLO
import torch
import gc
# torch.cuda.memory_summary(device=None, abbreviated=False)
# gc.collect()
torch.cuda.empty_cache()
print(torch.__version__)

model   = YOLO('yolo11s.pt')

results = model.train(
    data   = '/home/sowmya/AgSolaire/yolo_training.yaml',
    imgsz  = 600,
    epochs = 20,
    batch  = 5,
    name   = 'agsolaire_yolo_training',
    device = 0
   
)

model.val()

2.8.0+cu128
New https://pypi.org/project/ultralytics/8.3.200 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.199 🚀 Python-3.13.7 torch-2.8.0+cu128 CUDA:0 (NVIDIA T1200 Laptop GPU, 3904MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=5, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/sowmya/AgSolaire/yolo_training.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=600, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ag

In [9]:
# from ultralytics import YOLO

# Load the best weights from your training
# model = YOLO("/home/sowmya/AgSolaire/notebooks/yolo11s.pt")
# Replace with your test image path
image_path = "/home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg"

# Run prediction
# results = model.predict(source=image_path, conf=0.5, show=True)  # conf is confidence threshold
results = model(image_path)
# Get bounding boxes for the first image in batch
boxes = results[0].boxes

# Count seeds
num_seeds = len(boxes)
print(f"Number of seeds detected: {num_seeds}")
import cv2
cv2.namedWindow("YOLO ",cv2.WINDOW_NORMAL)
# Read the image
image = cv2.imread(image_path)

# Draw boxes
for box in boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 5)

cv2.putText(image, f"Seeds: {num_seeds}", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Save the result
cv2.imwrite("seed_detection_result.jpg", image)
cv2.imshow("YOLO ", image)
cv2.resizeWindow("YOLO ", 800,600)

cv2.waitKey(0)
cv2.destroyAllWindows()



image 1/1 /home/sowmya/AgSolaire/train_val_test_split/test/images/3-29.jpg: 608x480 38 seeds, 26.3ms
Speed: 1.9ms preprocess, 26.3ms inference, 0.9ms postprocess per image at shape (1, 3, 608, 480)
Number of seeds detected: 38
